In [1]:
import pandas as pd
import json
from os.path import join as pjoin
from os import listdir
from statistics import median

In [2]:
feature_preprocs = [
    "SKLFastICA",
    "SKLFeatureAgglomeration",
    "SKLKernelPCA",
    "SKLNystroem",
    "SKLPCA",
    "SKLPolynomialFeatures",
    "SKLRandomTreesEmbedding",
    "SKLRBFSampler",
    "SKLSelectFromExtraTrees",
    "SKLSelectFromLinearSVC",
    "SKLSelectPercentile",
    "SKLSelectRates"
]


scalers = [
    "SKLMinMaxScaler",
    "SKLNormalizer",
    "SKLPowerTransformer",
    "SKLQuantileTransformer",
    "SKLRobustScaler",
    "SKLStandardScaler"
]

imputers = [
    "SKLSimpleImpute"
    ]



In [3]:
def load_josn(path):
    with open(path, 'r') as f:
        return json.load(f)

In [4]:

def get_node_average_run_time_and_count(node_name, ds, mean=True):
    run_times = []

    dataset_path = pjoin("results", ds)
    for file in listdir(dataset_path):
        if file.endswith("run_time.json") and node_name in file:
            run_times.append(
                load_josn(pjoin(dataset_path, file))["total_seconds"])
            
    average = sum(run_times) / len(run_times)
    if mean:
        average = sum(run_times) / len(run_times)
    else:
        average = median(run_times)
    return average, len(run_times)

In [5]:
def get_saved_run_time_and_count_and_acutal_run_count(node_name, node_type , ds, ds_run_history_df, mean=True):
    avg_run_time, run_count  = get_node_average_run_time_and_count(node_name, ds, mean=mean)

    in_pipeline_count = ds_run_history_df[
        (ds_run_history_df[node_type] == node_name) &
        (ds_run_history_df["status"] =="success")
        ].shape[0]
    
    saved_run_time = (avg_run_time * in_pipeline_count) - (avg_run_time * run_count)
    saved_runs = in_pipeline_count - run_count

    return saved_run_time, saved_runs, run_count


    

In [6]:
def get_savings_and_actual_run_info(ds_name, mean=True):

    ds_rh = pd.read_csv(f"run_histories/{ds_name}_train_run_history.csv")

    saving = {}
    actual = {}

    saving["imputer_total_saved_time"] = 0
    saving["imputer_total_saved_runs"] = 0
    actual["imputer_actual_run_times"] = 0


    for imputer in imputers:
        imp_saved_time, imp_saved_runs, imp_run_count = get_saved_run_time_and_count_and_acutal_run_count(imputer, "imputer", ds_name, ds_rh, mean=mean)
        saving["imputer_total_saved_time"]  += imp_saved_time
        saving["imputer_total_saved_runs"] += imp_saved_runs
        actual["imputer_actual_run_times"] += imp_run_count


    saving["scaler_total_saved_sec"] = 0
    saving["scaler_total_saved_runs"] = 0
    actual["scaler_actual_run_times"] = 0

    for scaler in scalers:
        scaler_saved_time, scaler_saved_runs, scaler_run_count = get_saved_run_time_and_count_and_acutal_run_count(scaler, "scaler", ds_name, ds_rh, mean=mean)
        saving["scaler_total_saved_sec"] += scaler_saved_time
        saving["scaler_total_saved_runs"] += scaler_saved_runs
        actual["scaler_actual_run_times"] += scaler_run_count


    saving["feature_preproc_total_saved_sec"] = 0
    saving["feature_preproc_total_saved_runs"] = 0
    actual["feature_preproc_actual_run_times"] = 0

    for feature_preproc in feature_preprocs:
        fp_saved_time, fp_saved_runs, fp_run_count = get_saved_run_time_and_count_and_acutal_run_count(feature_preproc, "feature_preprocessor", ds_name, ds_rh, mean=mean)
        saving["feature_preproc_total_saved_sec"] += fp_saved_time
        saving["feature_preproc_total_saved_runs"] += fp_saved_runs
        actual["feature_preproc_actual_run_times"] += fp_run_count


    saving["total_saved_sec"] = saving["scaler_total_saved_sec"] + saving["imputer_total_saved_time"] + saving["feature_preproc_total_saved_sec"]
    saving["total_saved_runs"] = saving["scaler_total_saved_runs"] + saving["imputer_total_saved_runs"] + saving["feature_preproc_total_saved_runs"]

    return saving, actual


def get_total_run_sec(ds_name):
    return load_josn(f"logs/{ds_name}_time.json")["total_seconds"]




In [7]:
def make_summary_saving_dataframe(mean=True):

    dataset = []
    imputer_total_saved_time = []
    imputer_total_saved_runs = []
    imputer_total_runs = []

    scaler_total_saved_sec = []
    scaler_total_saved_runs = []
    scaler_total_runs = []

    feature_preproc_total_saved_sec = []
    feature_preproc_total_saved_runs = []
    feature_preproc_total_runs = []

    total_saved_sec = []
    total_saved_runs = []


    total_run_sec = []



    for ds in ["kc1", "pc4", "bank-marketing", "sylvine"]:
        
        dataset.append(ds)

        ds_savings, ds_actual = get_savings_and_actual_run_info(ds, mean=mean)
        imputer_total_saved_time.append(ds_savings["imputer_total_saved_time"])
        imputer_total_saved_runs.append(ds_savings["imputer_total_saved_runs"])

        scaler_total_saved_sec.append(ds_savings["scaler_total_saved_sec"])
        scaler_total_saved_runs.append(ds_savings["scaler_total_saved_runs"])
        
        feature_preproc_total_saved_sec.append(ds_savings["feature_preproc_total_saved_sec"])
        feature_preproc_total_saved_runs.append(ds_savings["feature_preproc_total_saved_runs"])

        total_saved_sec.append(ds_savings["total_saved_sec"])
        total_saved_runs.append(ds_savings["total_saved_runs"])

        total_run_sec.append(get_total_run_sec(ds))

        imputer_total_runs.append(ds_actual["imputer_actual_run_times"])
        scaler_total_runs.append(ds_actual["scaler_actual_run_times"])
        feature_preproc_total_runs.append(ds_actual["feature_preproc_actual_run_times"])




    df = pd.DataFrame()


    df["dataset"] = dataset
    df["imputer_total_runs"] = imputer_total_runs
    df["scaler_total_runs"] = scaler_total_runs
    df["feature_preproc_total_runs"] = feature_preproc_total_runs
    df["total_run_sec"] = total_run_sec



    df["imputer_total_saved_time"] = imputer_total_saved_time
    df["imputer_total_saved_runs"] = imputer_total_saved_runs

    df["scaler_total_saved_sec"] = scaler_total_saved_sec
    df["scaler_total_saved_runs"] = scaler_total_saved_runs

    df["feature_preproc_total_saved_sec"] = feature_preproc_total_saved_sec
    df["feature_preproc_total_saved_runs"] = feature_preproc_total_saved_runs

    df["total_saved_sec"] = total_saved_sec
    df["total_saved_runs"] = total_saved_runs

    df["time_saving_percent"] = df["total_saved_sec"] / df["total_run_sec"] * 100

    return df




In [8]:
make_summary_saving_dataframe(mean=True)

,dataset,imputer_total_runs,scaler_total_runs,feature_preproc_total_runs,total_run_sec,imputer_total_saved_time,imputer_total_saved_runs,scaler_total_saved_sec,scaler_total_saved_runs,feature_preproc_total_saved_sec,feature_preproc_total_saved_runs,total_saved_sec,total_saved_runs,time_saving_percent
0,kc1,1,6,84,138.432192,8.418351,1091,15.435092,930,49.315247,903,73.168691,2924,52.855257
1,pc4,1,6,84,248.137009,8.381583,1086,16.641448,930,55.242505,898,80.265535,2914,32.347265
2,bank-marketing,1,6,84,433.427274,7.842717,1091,10.933175,930,358.029099,903,376.804991,2924,86.936151
3,sylvine,1,6,84,443.256545,8.571891,1086,40.164781,930,107.216712,898,155.953384,2914,35.183549


In [9]:

make_summary_saving_dataframe(mean=False) # median is used instead of mean

,dataset,imputer_total_runs,scaler_total_runs,feature_preproc_total_runs,total_run_sec,imputer_total_saved_time,imputer_total_saved_runs,scaler_total_saved_sec,scaler_total_saved_runs,feature_preproc_total_saved_sec,feature_preproc_total_saved_runs,total_saved_sec,total_saved_runs,time_saving_percent
0,kc1,1,6,84,138.432192,8.418351,1091,15.435092,930,52.146315,903,75.999759,2924,54.900351
1,pc4,1,6,84,248.137009,8.381583,1086,16.641448,930,59.754679,898,84.777710,2914,34.165685
2,bank-marketing,1,6,84,433.427274,7.842717,1091,10.933175,930,346.002699,903,364.778591,2924,84.161430
3,sylvine,1,6,84,443.256545,8.571891,1086,40.164781,930,104.892317,898,153.628988,2914,34.659158
